# Mangrove Biomass Estimation: Thor Heyerdahl Climate Park\n\nSimplified working version for OGC demonstration

In [ ]:
import numpy as np\nimport pandas as pd\nimport plotly.graph_objects as go\nimport plotly.express as px\nfrom plotly.subplots import make_subplots

## Configuration & Data Generation

In [ ]:
# Configuration\nLOCATION = 'Thor Heyerdahl Climate Park, Myanmar'\nCENTER_LON, CENTER_LAT = 94.75, 16.75\nBOUNDS = {'west': 94.7, 'east': 94.8, 'south': 16.7, 'north': 16.8}\nGRID_SIZE = 30  # Reduced for performance\n\n# Create grid\nlons = np.linspace(BOUNDS['west'], BOUNDS['east'], GRID_SIZE)\nlats = np.linspace(BOUNDS['south'], BOUNDS['north'], GRID_SIZE)\nlon_grid, lat_grid = np.meshgrid(lons, lats)\n\n# Generate NDVI and biomass\nnp.random.seed(42)\ndistance = np.sqrt((lon_grid - CENTER_LON)**2 + (lat_grid - CENTER_LAT)**2)\nndvi = 0.75 * np.exp(-distance * 20) + 0.3\nndvi += 0.1 * np.sin(lon_grid * 200) * np.cos(lat_grid * 200)\nndvi += np.random.normal(0, 0.02, lon_grid.shape)\nndvi = np.clip(ndvi, 0.2, 0.9)\n\nbiomass = np.maximum(250.5 * ndvi - 75.2, 0)\n\nprint(f'Generated {GRID_SIZE}x{GRID_SIZE} grid = {GRID_SIZE**2} data points')

## Basic Map Visualization

In [ ]:
# Create DataFrame\ndf = pd.DataFrame({\n    'lon': lon_grid.flatten(),\n    'lat': lat_grid.flatten(),\n    'biomass': biomass.flatten(),\n    'ndvi': ndvi.flatten()\n})\n\n# Create scatter mapbox (correct syntax)\nfig = go.Figure()\n\nfig.add_trace(go.Scattermapbox(\n    lon=df['lon'],\n    lat=df['lat'],\n    mode='markers',\n    marker=dict(\n        size=10,\n        color=df['biomass'],\n        colorscale='Viridis',\n        showscale=True,\n        colorbar=dict(\n            title='Biomass<br>(Mg/ha)',\n            thickness=20\n        )\n    ),\n    text=[f'Biomass: {b:.1f} Mg/ha' for b in df['biomass']],\n    hovertemplate='%{text}<br>Lon: %{lon:.4f}<br>Lat: %{lat:.4f}<extra></extra>'\n))\n\n# Update layout with correct mapbox syntax\nfig.update_layout(\n    mapbox=dict(\n        style='open-street-map',\n        center=dict(lon=CENTER_LON, lat=CENTER_LAT),\n        zoom=10\n    ),\n    height=600,\n    title=f'Mangrove Biomass - {LOCATION}',\n    margin=dict(l=0, r=0, t=30, b=0)\n)\n\nfig.show()

## Heatmap Visualization

In [ ]:
# Create heatmap (2D grid view)\nfig = go.Figure(data=go.Heatmap(\n    x=lons,\n    y=lats,\n    z=biomass,\n    colorscale='Viridis',\n    colorbar=dict(title='Biomass (Mg/ha)')\n))\n\nfig.update_layout(\n    title='Biomass Distribution Heatmap',\n    xaxis_title='Longitude',\n    yaxis_title='Latitude',\n    height=500,\n    width=600\n)\n\nfig.show()

## 3D Surface Visualization

In [ ]:
# 3D surface plot\nfig = go.Figure(data=[go.Surface(\n    x=lons,\n    y=lats,\n    z=biomass,\n    colorscale='Viridis',\n    colorbar=dict(title='Biomass<br>(Mg/ha)')\n)])\n\nfig.update_layout(\n    title='3D Biomass Surface',\n    scene=dict(\n        xaxis_title='Longitude',\n        yaxis_title='Latitude',\n        zaxis_title='Biomass (Mg/ha)',\n        camera=dict(\n            eye=dict(x=1.5, y=-1.5, z=1.5)\n        )\n    ),\n    height=600\n)\n\nfig.show()

## Statistical Analysis Dashboard

In [ ]:
# Create subplots for analysis\nfig = make_subplots(\n    rows=2, cols=2,\n    subplot_titles=(\n        'Biomass Distribution',\n        'NDVI vs Biomass',\n        'Spatial Pattern',\n        'Latitude Profile'\n    ),\n    specs=[\n        [{'type': 'histogram'}, {'type': 'scatter'}],\n        [{'type': 'heatmap'}, {'type': 'scatter'}]\n    ]\n)\n\n# 1. Histogram\nfig.add_trace(\n    go.Histogram(x=df['biomass'], nbinsx=25, marker_color='green'),\n    row=1, col=1\n)\n\n# 2. NDVI vs Biomass scatter\nfig.add_trace(\n    go.Scatter(\n        x=df['ndvi'], y=df['biomass'],\n        mode='markers',\n        marker=dict(size=5, color='darkgreen', opacity=0.5)\n    ),\n    row=1, col=2\n)\n\n# 3. Spatial heatmap\nfig.add_trace(\n    go.Heatmap(x=lons, y=lats, z=biomass, colorscale='Viridis', showscale=False),\n    row=2, col=1\n)\n\n# 4. Latitude profile\nlat_means = df.groupby(pd.cut(df['lat'], 10))['biomass'].mean().reset_index()\nlat_means['lat_mid'] = lat_means['lat'].apply(lambda x: x.mid if hasattr(x, 'mid') else np.nan)\nlat_means = lat_means.dropna()\n\nfig.add_trace(\n    go.Scatter(\n        x=lat_means['lat_mid'], y=lat_means['biomass'],\n        mode='lines+markers',\n        line=dict(color='darkgreen', width=2)\n    ),\n    row=2, col=2\n)\n\n# Update layout\nfig.update_xaxes(title_text='Biomass (Mg/ha)', row=1, col=1)\nfig.update_xaxes(title_text='NDVI', row=1, col=2)\nfig.update_xaxes(title_text='Longitude', row=2, col=1)\nfig.update_xaxes(title_text='Latitude', row=2, col=2)\nfig.update_yaxes(title_text='Count', row=1, col=1)\nfig.update_yaxes(title_text='Biomass (Mg/ha)', row=1, col=2)\nfig.update_yaxes(title_text='Latitude', row=2, col=1)\nfig.update_yaxes(title_text='Mean Biomass', row=2, col=2)\n\nfig.update_layout(height=700, showlegend=False, title_text='Biomass Analysis Dashboard')\nfig.show()

## Summary Statistics

In [ ]:
# Calculate statistics\nstats = {\n    'Location': LOCATION,\n    'Center Coordinates': f'{CENTER_LAT:.3f}°N, {CENTER_LON:.3f}°E',\n    'Grid Points': len(df),\n    'Mean Biomass (Mg/ha)': df['biomass'].mean(),\n    'Max Biomass (Mg/ha)': df['biomass'].max(),\n    'Min Biomass (Mg/ha)': df['biomass'].min(),\n    'Std Dev (Mg/ha)': df['biomass'].std(),\n    'Mean NDVI': df['ndvi'].mean()\n}\n\n# Display as DataFrame\nstats_df = pd.DataFrame(list(stats.items()), columns=['Metric', 'Value'])\n\n# Format numeric values\nfor idx in range(3, len(stats_df)):\n    if isinstance(stats_df.loc[idx, 'Value'], (int, float)):\n        stats_df.loc[idx, 'Value'] = f'{stats_df.loc[idx, "Value"]:.2f}'\n\nprint('\\n=== MANGROVE BIOMASS SUMMARY ===\\n')\nfor _, row in stats_df.iterrows():\n    print(f'{row["Metric"]}: {row["Value"]}')

## Carbon Stock Estimation

In [ ]:
# Carbon calculations\narea_ha = 11000  # Approximate park area in hectares\nmean_biomass = df['biomass'].mean()\ntotal_biomass = mean_biomass * area_ha\ncarbon_fraction = 0.47  # Standard for mangroves\ncarbon_stock = total_biomass * carbon_fraction\nco2_equivalent = carbon_stock * 3.67  # CO2/C ratio\n\n# Create visualization\ncarbon_data = pd.DataFrame({\n    'Category': ['Total Biomass', 'Carbon Stock', 'CO₂ Equivalent'],\n    'Value': [total_biomass, carbon_stock, co2_equivalent],\n    'Unit': ['Mg', 'Mg C', 'Mg CO₂']\n})\n\nfig = go.Figure(data=[\n    go.Bar(\n        x=carbon_data['Category'],\n        y=carbon_data['Value'],\n        text=[f'{v:,.0f} {u}' for v, u in zip(carbon_data['Value'], carbon_data['Unit'])],\n        textposition='auto',\n        marker_color=['green', 'darkgreen', 'lightgreen']\n    )\n])\n\nfig.update_layout(\n    title='Carbon Stock Assessment',\n    yaxis_title='Amount (Mg)',\n    height=400\n)\n\nfig.show()\n\nprint(f'\\nEstimated carbon credit value (@$15/tCO₂): ${carbon_stock * 15:,.0f} USD')